In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

def scrape_tennis_rackets():
    # Set up request with random user agent
    ua = UserAgent()
    headers = {
        'User-Agent': ua.random,
        'Accept-Language': 'en-US,en;q=0.9',
        'Referer': 'https://www.google.com/'
    }
    
    try:
        print("Sending request to the website...")
        response = requests.get(
            "https://www.tenniswarehouse-europe.com/TennisRackets.html",
            headers=headers,
            timeout=10
        )
        print(f"Response status code: {response.status_code}")
        
        # Check if request was successful
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code}")
            return pd.DataFrame()
        
        # Parse HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        print("HTML parsed successfully")
        
        # Debug: Save HTML to file for inspection
        with open('tennis_page.html', 'w', encoding='utf-8') as f:
            f.write(response.text)
        print("Saved page HTML to tennis_page.html for inspection")
        
        # Find all racket products - updated selector
        products = soup.select('div.product-item-info')
        print(f"Found {len(products)} products")
        
        if not products:
            print("No products found - check HTML structure")
            return pd.DataFrame()
        
        # Initialize list to store racket data
        rackets_data = []
        
        for product in products:
            try:
                racket = {}
                
                # Extract brand and model
                name_element = product.select_one('a.product-item-link')
                if name_element:
                    full_name = name_element.get_text(strip=True)
                    racket['brand'] = full_name.split(' ', 1)[0]
                    racket['model'] = full_name.split(' ', 1)[1] if ' ' in full_name else ''
                
                # Extract prices
                price_element = product.select_one('span.price')
                if price_element:
                    racket['price'] = price_element.get_text(strip=True)
                
                old_price_element = product.select_one('span.old-price span.price')
                if old_price_element:
                    racket['original_price'] = old_price_element.get_text(strip=True)
                
                # Extract specifications - may need to visit individual product pages
                # For now, let's get what we can from the listing
                spec_elements = product.select('div.product-specs li')
                for spec in spec_elements:
                    text = spec.get_text(strip=True)
                    if 'Head Size:' in text:
                        racket['head_size'] = text.replace('Head Size:', '').strip()
                    elif 'Length:' in text:
                        racket['length'] = text.replace('Length:', '').strip()
                    # Add other spec fields similarly
                
                rackets_data.append(racket)
            
            except Exception as e:
                print(f"Error processing product: {e}")
                continue
        
        # Create DataFrame
        df = pd.DataFrame(rackets_data)
        
        if not df.empty:
            # Clean data
            for col in ['price', 'original_price']:
                if col in df.columns:
                    df[col] = df[col].str.replace('[^\d.]', '', regex=True)
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            print("Successfully scraped data:")
            print(df.head())
        else:
            print("Created DataFrame but it's empty")
        
        return df
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# Run the scraper
tennis_rackets_df = scrape_tennis_rackets()

if not tennis_rackets_df.empty:
    tennis_rackets_df.to_csv('tennis_rackets_data.csv', index=False)
    print("Data saved to tennis_rackets_data.csv")
else:
    print("No data was scraped")

<>:93: SyntaxWarning: invalid escape sequence '\d'
<>:93: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Anne\AppData\Local\Temp\ipykernel_13204\301165937.py:93: SyntaxWarning: invalid escape sequence '\d'
  df[col] = df[col].str.replace('[^\d.]', '', regex=True)


Sending request to the website...
Response status code: 200
HTML parsed successfully
Saved page HTML to tennis_page.html for inspection
Found 0 products
No products found - check HTML structure
No data was scraped


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)
driver.get("https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html")
soup = BeautifulSoup(driver.page_source, 'html.parser')
# Continue with scraping...

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_racket_details(url):
    # Configure Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize driver (you'll need chromedriver installed)
    service = Service(executable_path='chromedriver')  # Update path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        print(f"Accessing {url}")
        driver.get(url)
        
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-info-main"))
        print("Page loaded successfully")
        
        # Initialize data dictionary
        racket_data = {
            'brand': 'Yonex',
            'model': None,
            'price': None,
            'original_price': None,
            'head_size': None,
            'length': None,
            'strung_weight': None,
            'unstrung_weight': None,
            'swingweight': None,
            'stiffness': None,
            'beam_width': None,
            'color': None,
            'string_pattern': None,
            'tension': None,
            'grip_size': None,
            'balance': None
        }
        
        # Extract model name
        try:
            model_element = driver.find_element(By.CSS_SELECTOR, "h1.page-title span.base")
            racket_data['model'] = model_element.text.strip()
        except Exception as e:
            print(f"Could not extract model: {e}")
        
        # Extract prices
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, "span.price-final_price span.price")
            racket_data['price'] = price_element.text.strip()
            
            # Check for original price
            try:
                original_price = driver.find_element(By.CSS_SELECTOR, "span.old-price span.price")
                racket_data['original_price'] = original_price.text.strip()
            except:
                pass
        except Exception as e:
            print(f"Could not extract price: {e}")
        
        # Extract specifications from the table
        try:
            spec_rows = driver.find_elements(By.CSS_SELECTOR, "table.data.table.additional-attributes tbody tr")
            
            for row in spec_rows:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                
                if 'Head Size' in th:
                    racket_data['head_size'] = td
                elif 'Length' in th:
                    racket_data['length'] = td
                elif 'Strung Weight' in th:
                    racket_data['strung_weight'] = td
                elif 'Unstrung Weight' in th:
                    racket_data['unstrung_weight'] = td
                elif 'Swingweight' in th:
                    racket_data['swingweight'] = td
                elif 'Stiffness' in th:
                    racket_data['stiffness'] = td
                elif 'Beam Width' in th:
                    racket_data['beam_width'] = td
                elif 'Colour' in th:
                    racket_data['color'] = td
                elif 'String Pattern' in th:
                    racket_data['string_pattern'] = td
                elif 'Tension' in th:
                    racket_data['tension'] = td
                elif 'Grip Size' in th:
                    racket_data['grip_size'] = td
                elif 'Balance' in th:
                    racket_data['balance'] = td
                    
        except Exception as e:
            print(f"Could not extract specifications: {e}")
        
        # Create DataFrame
        df = pd.DataFrame([racket_data])
        
        # Clean price columns
        for col in ['price', 'original_price']:
            if col in df.columns:
                df[col] = df[col].str.replace('[^\d.,]', '', regex=True)
                df[col] = df[col].str.replace(',', '.').astype(float)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()

# URL to scrape
racket_url = "https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html"

# Run the scraper
racket_df = scrape_racket_details(racket_url)

if not racket_df.empty:
    print("\nScraped Racket Data:")
    print(racket_df)
    racket_df.to_csv('yonex_ezone_racket.csv', index=False)
    print("\nData saved to yonex_ezone_racket.csv")
else:
    print("Failed to scrape racket data")

SyntaxError: '(' was never closed (1473522330.py, line 26)

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_racket_details(url):
    # Configure Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize driver (you'll need chromedriver installed)
    service = Service(executable_path='chromedriver')  # Update path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        print(f"Accessing {url}")
        driver.get(url)
        
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-info-main"))
        print("Page loaded successfully")
        
        # Initialize data dictionary
        racket_data = {
            'brand': 'Yonex',
            'model': None,
            'price': None,
            'original_price': None,
            'head_size': None,
            'length': None,
            'strung_weight': None,
            'unstrung_weight': None,
            'swingweight': None,
            'stiffness': None,
            'beam_width': None,
            'color': None,
            'string_pattern': None,
            'tension': None,
            'grip_size': None,
            'balance': None
        }
        
        # Extract model name
        try:
            model_element = driver.find_element(By.CSS_SELECTOR, "h1.page-title span.base")
            racket_data['model'] = model_element.text.strip()
        except Exception as e:
            print(f"Could not extract model: {e}")
        
        # Extract prices
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, "span.price-final_price span.price")
            racket_data['price'] = price_element.text.strip()
            
            # Check for original price
            try:
                original_price = driver.find_element(By.CSS_SELECTOR, "span.old-price span.price")
                racket_data['original_price'] = original_price.text.strip()
            except:
                pass
        except Exception as e:
            print(f"Could not extract price: {e}")
        
        # Extract specifications from the table
        try:
            spec_rows = driver.find_elements(By.CSS_SELECTOR, "table.data.table.additional-attributes tbody tr")
            
            for row in spec_rows:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                
                if 'Head Size' in th:
                    racket_data['head_size'] = td
                elif 'Length' in th:
                    racket_data['length'] = td
                elif 'Strung Weight' in th:
                    racket_data['strung_weight'] = td
                elif 'Unstrung Weight' in th:
                    racket_data['unstrung_weight'] = td
                elif 'Swingweight' in th:
                    racket_data['swingweight'] = td
                elif 'Stiffness' in th:
                    racket_data['stiffness'] = td
                elif 'Beam Width' in th:
                    racket_data['beam_width'] = td
                elif 'Colour' in th:
                    racket_data['color'] = td
                elif 'String Pattern' in th:
                    racket_data['string_pattern'] = td
                elif 'Tension' in th:
                    racket_data['tension'] = td
                elif 'Grip Size' in th:
                    racket_data['grip_size'] = td
                elif 'Balance' in th:
                    racket_data['balance'] = td
                    
        except Exception as e:
            print(f"Could not extract specifications: {e}")
        
        # Create DataFrame
        df = pd.DataFrame([racket_data])
        
        # Clean price columns
        for col in ['price', 'original_price']:
            if col in df.columns:
                df[col] = df[col].str.replace('[^\d.,]', '', regex=True)
                df[col] = df[col].str.replace(',', '.').astype(float)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()

# URL to scrape
racket_url = "https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html"

# Run the scraper
racket_df = scrape_racket_details(racket_url)

if not racket_df.empty:
    print("\nScraped Racket Data:")
    print(racket_df)
    racket_df.to_csv('yonex_ezone_racket.csv', index=False)
    print("\nData saved to yonex_ezone_racket.csv")
else:
    print("Failed to scrape racket data")

SyntaxError: '(' was never closed (1473522330.py, line 26)

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_racket_details(url):
    # Configure Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize driver (you'll need chromedriver installed)
    service = Service(executable_path='chromedriver')  # Update path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        print(f"Accessing {url}")
        driver.get(url)
        
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-info-main"))
        print("Page loaded successfully")
        
        # Initialize data dictionary
        racket_data = {
            'brand': 'Yonex',
            'model': None,
            'price': None,
            'original_price': None,
            'head_size': None,
            'length': None,
            'strung_weight': None,
            'unstrung_weight': None,
            'swingweight': None,
            'stiffness': None,
            'beam_width': None,
            'color': None,
            'string_pattern': None,
            'tension': None,
            'grip_size': None,
            'balance': None
        }
        
        # Extract model name
        try:
            model_element = driver.find_element(By.CSS_SELECTOR, "h1.page-title span.base")
            racket_data['model'] = model_element.text.strip()
        except Exception as e:
            print(f"Could not extract model: {e}")
        
        # Extract prices
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, "span.price-final_price span.price")
            racket_data['price'] = price_element.text.strip()
            
            # Check for original price
            try:
                original_price = driver.find_element(By.CSS_SELECTOR, "span.old-price span.price")
                racket_data['original_price'] = original_price.text.strip()
            except:
                pass
        except Exception as e:
            print(f"Could not extract price: {e}")
        
        # Extract specifications from the table
        try:
            spec_rows = driver.find_elements(By.CSS_SELECTOR, "table.data.table.additional-attributes tbody tr")
            
            for row in spec_rows:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                
                if 'Head Size' in th:
                    racket_data['head_size'] = td
                elif 'Length' in th:
                    racket_data['length'] = td
                elif 'Strung Weight' in th:
                    racket_data['strung_weight'] = td
                elif 'Unstrung Weight' in th:
                    racket_data['unstrung_weight'] = td
                elif 'Swingweight' in th:
                    racket_data['swingweight'] = td
                elif 'Stiffness' in th:
                    racket_data['stiffness'] = td
                elif 'Beam Width' in th:
                    racket_data['beam_width'] = td
                elif 'Colour' in th:
                    racket_data['color'] = td
                elif 'String Pattern' in th:
                    racket_data['string_pattern'] = td
                elif 'Tension' in th:
                    racket_data['tension'] = td
                elif 'Grip Size' in th:
                    racket_data['grip_size'] = td
                elif 'Balance' in th:
                    racket_data['balance'] = td
                    
        except Exception as e:
            print(f"Could not extract specifications: {e}")
        
        # Create DataFrame
        df = pd.DataFrame([racket_data])
        
        # Clean price columns
        for col in ['price', 'original_price']:
            if col in df.columns:
                df[col] = df[col].str.replace('[^\d.,]', '', regex=True)
                df[col] = df[col].str.replace(',', '.').astype(float)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()

# URL to scrape
racket_url = "https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html"

# Run the scraper
racket_df = scrape_racket_details(racket_url)

if not racket_df.empty:
    print("\nScraped Racket Data:")
    print(racket_df)
    racket_df.to_csv('yonex_ezone_racket.csv', index=False)
    print("\nData saved to yonex_ezone_racket.csv")
else:
    print("Failed to scrape racket data")

SyntaxError: '(' was never closed (1473522330.py, line 26)

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def scrape_racket_details(url):
    # Configure Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize driver (you'll need chromedriver installed)
    service = Service(executable_path='chromedriver')  # Update path if needed
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        print(f"Accessing {url}")
        driver.get(url)
        
        # Wait for page to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-info-main"))
        print("Page loaded successfully")
        
        # Initialize data dictionary
        racket_data = {
            'brand': 'Yonex',
            'model': None,
            'price': None,
            'original_price': None,
            'head_size': None,
            'length': None,
            'strung_weight': None,
            'unstrung_weight': None,
            'swingweight': None,
            'stiffness': None,
            'beam_width': None,
            'color': None,
            'string_pattern': None,
            'tension': None,
            'grip_size': None,
            'balance': None
        }
        
        # Extract model name
        try:
            model_element = driver.find_element(By.CSS_SELECTOR, "h1.page-title span.base")
            racket_data['model'] = model_element.text.strip()
        except Exception as e:
            print(f"Could not extract model: {e}")
        
        # Extract prices
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, "span.price-final_price span.price")
            racket_data['price'] = price_element.text.strip()
            
            # Check for original price
            try:
                original_price = driver.find_element(By.CSS_SELECTOR, "span.old-price span.price")
                racket_data['original_price'] = original_price.text.strip()
            except:
                pass
        except Exception as e:
            print(f"Could not extract price: {e}")
        
        # Extract specifications from the table
        try:
            spec_rows = driver.find_elements(By.CSS_SELECTOR, "table.data.table.additional-attributes tbody tr")
            
            for row in spec_rows:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                
                if 'Head Size' in th:
                    racket_data['head_size'] = td
                elif 'Length' in th:
                    racket_data['length'] = td
                elif 'Strung Weight' in th:
                    racket_data['strung_weight'] = td
                elif 'Unstrung Weight' in th:
                    racket_data['unstrung_weight'] = td
                elif 'Swingweight' in th:
                    racket_data['swingweight'] = td
                elif 'Stiffness' in th:
                    racket_data['stiffness'] = td
                elif 'Beam Width' in th:
                    racket_data['beam_width'] = td
                elif 'Colour' in th:
                    racket_data['color'] = td
                elif 'String Pattern' in th:
                    racket_data['string_pattern'] = td
                elif 'Tension' in th:
                    racket_data['tension'] = td
                elif 'Grip Size' in th:
                    racket_data['grip_size'] = td
                elif 'Balance' in th:
                    racket_data['balance'] = td
                    
        except Exception as e:
            print(f"Could not extract specifications: {e}")
        
        # Create DataFrame
        df = pd.DataFrame([racket_data])
        
        # Clean price columns
        for col in ['price', 'original_price']:
            if col in df.columns:
                df[col] = df[col].str.replace('[^\d.,]', '', regex=True)
                df[col] = df[col].str.replace(',', '.').astype(float)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()

# URL to scrape
racket_url = "https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html"

# Run the scraper
racket_df = scrape_racket_details(racket_url)

if not racket_df.empty:
    print("\nScraped Racket Data:")
    print(racket_df)
    racket_df.to_csv('yonex_ezone_racket.csv', index=False)
    print("\nData saved to yonex_ezone_racket.csv")
else:
    print("Failed to scrape racket data")

SyntaxError: '(' was never closed (1473522330.py, line 26)

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

def scrape_racket_details(url):
    # Configure Selenium
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    # Initialize driver with automatic driver management
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    try:
        print(f"Accessing {url}")
        driver.get(url)
        
        # Wait for page to load - FIXED SYNTAX HERE
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "product-info-main"))
        print("Page loaded successfully")
        
        # Initialize data dictionary
        racket_data = {
            'brand': 'Yonex',
            'model': None,
            'price': None,
            'original_price': None,
            'head_size': None,
            'length': None,
            'strung_weight': None,
            'unstrung_weight': None,
            'swingweight': None,
            'stiffness': None,
            'beam_width': None,
            'color': None,
            'string_pattern': None,
            'tension': None,
            'grip_size': None,
            'balance': None
        }
        
        # Extract model name
        try:
            model_element = driver.find_element(By.CSS_SELECTOR, "h1.page-title span.base")
            racket_data['model'] = model_element.text.strip()
        except Exception as e:
            print(f"Could not extract model: {e}")
        
        # Extract prices
        try:
            price_element = driver.find_element(By.CSS_SELECTOR, "span.price-final_price span.price")
            racket_data['price'] = price_element.text.strip()
            
            # Check for original price
            try:
                original_price = driver.find_element(By.CSS_SELECTOR, "span.old-price span.price")
                racket_data['original_price'] = original_price.text.strip()
            except:
                pass
        except Exception as e:
            print(f"Could not extract price: {e}")
        
        # Extract specifications from the table
        try:
            spec_rows = driver.find_elements(By.CSS_SELECTOR, "table.data.table.additional-attributes tbody tr")
            
            for row in spec_rows:
                th = row.find_element(By.TAG_NAME, "th").text.strip()
                td = row.find_element(By.TAG_NAME, "td").text.strip()
                
                if 'Head Size' in th:
                    racket_data['head_size'] = td
                elif 'Length' in th:
                    racket_data['length'] = td
                elif 'Strung Weight' in th:
                    racket_data['strung_weight'] = td
                elif 'Unstrung Weight' in th:
                    racket_data['unstrung_weight'] = td
                elif 'Swingweight' in th:
                    racket_data['swingweight'] = td
                elif 'Stiffness' in th:
                    racket_data['stiffness'] = td
                elif 'Beam Width' in th:
                    racket_data['beam_width'] = td
                elif 'Colour' in th:
                    racket_data['color'] = td
                elif 'String Pattern' in th:
                    racket_data['string_pattern'] = td
                elif 'Tension' in th:
                    racket_data['tension'] = td
                elif 'Grip Size' in th:
                    racket_data['grip_size'] = td
                elif 'Balance' in th:
                    racket_data['balance'] = td
                    
        except Exception as e:
            print(f"Could not extract specifications: {e}")
        
        # Create DataFrame
        df = pd.DataFrame([racket_data])
        
        # Clean price columns
        for col in ['price', 'original_price']:
            if col in df.columns:
                df[col] = df[col].str.replace('[^\d.,]', '', regex=True)
                df[col] = df[col].str.replace(',', '.').astype(float)
        
        return df
    
    except Exception as e:
        print(f"Error during scraping: {e}")
        return pd.DataFrame()
    
    finally:
        driver.quit()

# URL to scrape
racket_url = "https://www.tenniswarehouse-europe.com/Yonex_EZONE_98_Tour_315g_Blast_Blue_Racket/descpageRCYONEX-EZ98TB-EN.html"

# Run the scraper
racket_df = scrape_racket_details(racket_url)

if not racket_df.empty:
    print("\nScraped Racket Data:")
    print(racket_df)
    racket_df.to_csv('yonex_ezone_racket.csv', index=False)
    print("\nData saved to yonex_ezone_racket.csv")
else:
    print("Failed to scrape racket data")

SyntaxError: '(' was never closed (521416936.py, line 26)